# Analysis for RNAseq

This notebook contains differential expression and gene set enrichment analysis for RNA-sequencing performed in KP4 cells with DOX-mediated knockdown of PELO alongside either Ch2 or FOCAD knockout.  Expected counts from RSEM are inputted to DESeq2, with GSEA pre-rank run on the shrunken LFCs for the Hallmark collection of gene sets.

In [1]:
import numpy as np
import pandas as pd

from helper_funcs import *

from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats

import gseapy as gp

# Import data

In [2]:
redownload = False
metadata = read_in_data("sample_metadata_RNAseq", index_col=0, redownload=redownload)
raw_counts_df = read_in_data("RSEM_expected_count_RNAseq", index_col=0, redownload=redownload)
gene_symbol_mapping = read_in_data("HGNC_complete_set", redownload=redownload, low_memory=False)
hallmarks = read_in_data("hallmark_gene_sets_2023", index_col=0, redownload=redownload)["Hallmark Gene Set"].str.split().to_dict()

Fetching local copy of sample_metadata_RNAseq
Fetching local copy of RSEM_expected_count_RNAseq
Fetching local copy of HGNC_complete_set
Fetching local copy of hallmark_gene_sets_2023


# Rename to gene symbols

In [3]:
#remove version suffix on RSEM output
gene_map = pd.Series(raw_counts_df.columns.str.split(".").str[0], index=raw_counts_df.columns)
#map to gene symbol
gene_map = gene_map.to_frame("stripped").reset_index().merge(
    gene_symbol_mapping, left_on="stripped", right_on="ensembl_gene_id", how="inner"
).set_index("index")["symbol"]
#drop ensembl IDs that map to multiple genes
gene_map = gene_map.loc[~gene_map.index.duplicated(keep=False)]
raw_counts_df = raw_counts_df.groupby(gene_map, axis=1).sum()
raw_counts_df.shape

(12, 41098)

# Filter out genes with no counts in any condition.

In [4]:
genes_to_keep = raw_counts_df.columns[raw_counts_df.sum() > 1]
counts_df = raw_counts_df.loc[:, genes_to_keep].copy()
counts_df.shape

(12, 20130)

# Run Inference

In [5]:
def run_deseq2(comps, ref_level):
    #setup DEseq2 with condition design factor and designated reference
    inference = DefaultInference(n_cpus=8)
    dds = DeseqDataSet(
        counts=counts_df,
        metadata=metadata,
        design_factors="Condition",
        ref_level=["Condition", ref_level],
        refit_cooks=True,
        inference=inference,
    )
    dds.deseq2()

    #iterate through comparisons to the reference
    results, pre_dfs = {}, {}
    for name, contrast in comps.items():
        #run diff expr calling
        stats = DeseqStats(dds, contrast=contrast, inference=inference)
        stats.summary()
        #shrink LFCs
        stats.lfc_shrink()
        #store results
        results_df = stats.results_df
        results_df["-log10(padj)"] = -np.log10(results_df["padj"])
        results_df["UPR"] = results_df.index.isin(hallmarks["HALLMARK_UNFOLDED_PROTEIN_RESPONSE"])

        #run GSEA prerank on shrunken LFCs
        pre_res = gp.prerank(
            results_df["log2FoldChange"], gene_sets=hallmarks,
            threads=4, min_size=5, max_size=1000, permutation_num=1000,
            outdir=None, seed=6, verbose=True,
        )

        #store GSEA outputs
        pre_df = pd.concat({
            k:pd.Series(pre_res.results[k]) for k in pre_res.results.keys()
        }, axis=1).T.rename_axis("gene set").reset_index().set_index(
            ["gene set", "nes", "pval", "fdr"]
        )["RES"].apply(pd.Series).reset_index().set_index('gene set').T

        results[name] = results_df
        pre_dfs[name] = pre_df

    return results, pre_dfs

# Call differentially expressed genes

In [6]:
comps = {
    "DOX- with FOCAD KO vs Ch2 KO":["Condition", "FOCAD KO + DoxNeg", "Chr2 KO + DoxNeg"],
    "Ch2 KO with DOX+ (PELO KD) vs DOX-":["Condition", "Chr2 KO + DoxPos", "Chr2 KO + DoxNeg"],
}
results_ctrl, pre_df_ctrl = run_deseq2(comps, "Chr2 KO + DoxNeg")

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 0.74 seconds.

Fitting dispersion trend curve...
... done in 0.91 seconds.

Fitting MAP dispersions...
... done in 0.86 seconds.

Fitting LFCs...
... done in 0.67 seconds.

Replacing 0 outlier genes.

Running Wald tests...
... done in 0.81 seconds.

Fitting MAP LFCs...


Log2 fold change & Wald test p-value: Condition FOCAD KO + DoxNeg vs Chr2 KO + DoxNeg
            baseMean  log2FoldChange     lfcSE      stat    pvalue      padj
symbol                                                                      
A1BG        0.380526        0.365326  4.707328  0.077608  0.938140       NaN
A2M         0.090201        1.418620  4.568222  0.310541  0.756150       NaN
A2M-AS1     0.723841       -0.667839  2.146901 -0.311071  0.755747       NaN
A2MP1       0.103156       -0.764528  4.630581 -0.165104  0.868862       NaN
A4GALT     75.024235        0.075366  0.218764  0.344507  0.730465  0.967868
...              ...             ...       ...       ...       ...       ...
ZYG11A      0.709035       -0.403307  2.056229 -0.196139  0.844501       NaN
ZYG11B    473.775645       -0.030182  0.113323 -0.266339  0.789978  0.977403
ZYX      3130.500188        0.039010  0.068009  0.573608  0.566233  0.945367
ZZEF1     769.625457        0.171289  0.091868  1.864513  0.062250 

... done in 1.97 seconds.

2024-08-01 11:34:04,514 [WARNING] Duplicated values found in preranked stats: 9.21% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
2024-08-01 11:34:04,514 [INFO] Parsing data files for GSEA.............................
2024-08-01 11:34:04,516 [INFO] 0000 gene_sets have been filtered out when max_size=1000 and min_size=5
2024-08-01 11:34:04,517 [INFO] 0050 gene_sets used for further statistical testing.....
2024-08-01 11:34:04,517 [INFO] Start to run GSEA...Might take a while..................


Shrunk log2 fold change & Wald test p-value: Condition FOCAD KO + DoxNeg vs Chr2 KO + DoxNeg
            baseMean  log2FoldChange     lfcSE      stat    pvalue      padj
symbol                                                                      
A1BG        0.380526       -0.000264  0.058204  0.077608  0.938140       NaN
A2M         0.090201        0.000216  0.054060  0.310541  0.756150       NaN
A2M-AS1     0.723841       -0.000521  0.046723 -0.311071  0.755747       NaN
A2MP1       0.103156       -0.000328  0.064668 -0.165104  0.868862       NaN
A4GALT     75.024235        0.004383  0.044394  0.344507  0.730465  0.967868
...              ...             ...       ...       ...       ...       ...
ZYG11A      0.709035       -0.000320  0.050717 -0.196139  0.844501       NaN
ZYG11B    473.775645       -0.005640  0.041980 -0.266339  0.789978  0.977403
ZYX      3130.500188        0.014759  0.038574  0.573608  0.566233  0.945367
ZZEF1     769.625457        0.063070  0.075787  1.864513  0.

2024-08-01 11:34:05,993 [INFO] Congratulations. GSEApy runs successfully................

Running Wald tests...
... done in 0.57 seconds.

Fitting MAP LFCs...
/opt/anaconda3/envs/pelo/lib/python3.9/site-packages/pydeseq2/utils.py:1249: RuntimeWarning: overflow encountered in exp
  counts - (counts + size) / (1 + size * np.exp(-xbeta - offset))


Log2 fold change & Wald test p-value: Condition Chr2 KO + DoxPos vs Chr2 KO + DoxNeg
            baseMean  log2FoldChange     lfcSE      stat    pvalue      padj
symbol                                                                      
A1BG        0.380526        0.293182  4.698443  0.062400  0.950245  0.998403
A2M         0.090201        0.432979  4.749863  0.091156  0.927369  0.998403
A2M-AS1     0.723841       -0.652519  2.128984 -0.306493  0.759229  0.998403
A2MP1       0.103156       -0.836673  4.621317 -0.181046  0.856331  0.998403
A4GALT     75.024235        0.169986  0.215607  0.788408  0.430458  0.998403
...              ...             ...       ...       ...       ...       ...
ZYG11A      0.709035       -2.294694  2.614536 -0.877668  0.380124  0.998403
ZYG11B    473.775645        0.004060  0.112471  0.036097  0.971205  0.998403
ZYX      3130.500188        0.091904  0.067709  1.357336  0.174675  0.998403
ZZEF1     769.625457       -0.076446  0.092422 -0.827141  0.408157  

/opt/anaconda3/envs/pelo/lib/python3.9/site-packages/pydeseq2/utils.py:1249: RuntimeWarning: overflow encountered in exp
  counts - (counts + size) / (1 + size * np.exp(-xbeta - offset))
/opt/anaconda3/envs/pelo/lib/python3.9/site-packages/pydeseq2/utils.py:1249: RuntimeWarning: overflow encountered in exp
  counts - (counts + size) / (1 + size * np.exp(-xbeta - offset))
/opt/anaconda3/envs/pelo/lib/python3.9/site-packages/pydeseq2/utils.py:1249: RuntimeWarning: overflow encountered in exp
  counts - (counts + size) / (1 + size * np.exp(-xbeta - offset))
/opt/anaconda3/envs/pelo/lib/python3.9/site-packages/pydeseq2/utils.py:1249: RuntimeWarning: overflow encountered in exp
  counts - (counts + size) / (1 + size * np.exp(-xbeta - offset))
/opt/anaconda3/envs/pelo/lib/python3.9/site-packages/pydeseq2/utils.py:1249: RuntimeWarning: overflow encountered in exp
  counts - (counts + size) / (1 + size * np.exp(-xbeta - offset))
/opt/anaconda3/envs/pelo/lib/python3.9/site-packages/pydeseq2/uti

Shrunk log2 fold change & Wald test p-value: Condition Chr2 KO + DoxPos vs Chr2 KO + DoxNeg
            baseMean  log2FoldChange     lfcSE      stat    pvalue      padj
symbol                                                                      
A1BG        0.380526   -9.626434e-08  0.001100  0.062400  0.950245  0.998403
A2M         0.090201   -7.735856e-08  0.001118  0.091156  0.927369  0.998403
A2M-AS1     0.723841   -1.609881e-07  0.000881 -0.306493  0.759229  0.998403
A2MP1       0.103156   -1.219659e-07  0.001224 -0.181046  0.856331  0.998403
A4GALT     75.024235    3.830802e-06  0.000849  0.788408  0.430458  0.998403
...              ...             ...       ...       ...       ...       ...
ZYG11A      0.709035   -5.357074e-07  0.000944 -0.877668  0.380124  0.998403
ZYG11B    473.775645    1.931635e-07  0.000845  0.036097  0.971205  0.998403
ZYX      3130.500188    2.208994e-05  0.000834  1.357336  0.174675  0.998403
ZZEF1     769.625457   -8.943421e-06  0.000848 -0.827141  0.4

2024-08-01 11:34:11,864 [INFO] Congratulations. GSEApy runs successfully................



In [7]:
comps = {
    "FOCAD KO with DOX+ (PELO KD) vs DOX-":["Condition", "FOCAD KO + DoxPos", "FOCAD KO + DoxNeg"],
}
results_focad, pre_df_focad = run_deseq2(comps, "FOCAD KO + DoxNeg")

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 0.96 seconds.

Fitting dispersion trend curve...
... done in 3.96 seconds.

Fitting MAP dispersions...
... done in 0.97 seconds.

Fitting LFCs...
... done in 0.99 seconds.

Replacing 0 outlier genes.

Running Wald tests...
... done in 0.68 seconds.

Fitting MAP LFCs...


Log2 fold change & Wald test p-value: Condition FOCAD KO + DoxPos vs FOCAD KO + DoxNeg
            baseMean  log2FoldChange     lfcSE      stat    pvalue      padj
symbol                                                                      
A1BG        0.380526        2.413093  4.585117  0.526288  0.598688       NaN
A2M         0.090201       -0.928780  4.759442 -0.195145  0.845280       NaN
A2M-AS1     0.723841       -0.598816  2.486310 -0.240845  0.809675       NaN
A2MP1       0.103156       -0.015306  4.942710 -0.003097  0.997529       NaN
A4GALT     75.024235        0.175495  0.231850  0.756931  0.449091  0.754089
...              ...             ...       ...       ...       ...       ...
ZYG11A      0.709035        0.349940  2.180216  0.160507  0.872482       NaN
ZYG11B    473.775645        0.106453  0.118058  0.901698  0.367217  0.690929
ZYX      3130.500188        0.104748  0.069194  1.513832  0.130068  0.423030
ZZEF1     769.625457       -0.126508  0.095676 -1.322250  0.186085

... done in 1.78 seconds.

2024-08-01 11:34:23,608 [WARNING] Duplicated values found in preranked stats: 9.21% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
2024-08-01 11:34:23,609 [INFO] Parsing data files for GSEA.............................
2024-08-01 11:34:23,612 [INFO] 0000 gene_sets have been filtered out when max_size=1000 and min_size=5
2024-08-01 11:34:23,613 [INFO] 0050 gene_sets used for further statistical testing.....
2024-08-01 11:34:23,614 [INFO] Start to run GSEA...Might take a while..................


Shrunk log2 fold change & Wald test p-value: Condition FOCAD KO + DoxPos vs FOCAD KO + DoxNeg
            baseMean  log2FoldChange     lfcSE      stat    pvalue      padj
symbol                                                                      
A1BG        0.380526        0.002769  0.107375  0.526288  0.598688       NaN
A2M         0.090201       -0.001745  0.140875 -0.195145  0.845280       NaN
A2M-AS1     0.723841       -0.000969  0.110595 -0.240845  0.809675       NaN
A2MP1       0.103156       -0.000199  0.151709 -0.003097  0.997529       NaN
A4GALT     75.024235        0.044734  0.103839  0.756931  0.449091  0.754089
...              ...             ...       ...       ...       ...       ...
ZYG11A      0.709035        0.001735  0.115877  0.160507  0.872482       NaN
ZYG11B    473.775645        0.061559  0.085918  0.901698  0.367217  0.690929
ZYX      3130.500188        0.086237  0.062616  1.513832  0.130068  0.423030
ZZEF1     769.625457       -0.087837  0.080622 -1.322250  0

2024-08-01 11:34:25,032 [INFO] Congratulations. GSEApy runs successfully................



# Save outputs

In [8]:
results_ctrl["DOX- with FOCAD KO vs Ch2 KO"].to_csv("outputs/DOXneg_FOCAD_vs_Ch2_KO_deseq2.csv")
pre_df_ctrl["DOX- with FOCAD KO vs Ch2 KO"].to_csv("outputs/DOXneg_FOCAD_vs_Ch2_KO_gsea_prerank.csv")

results_ctrl["Ch2 KO with DOX+ (PELO KD) vs DOX-"].to_csv("outputs/Ch2_KO_DOXpos_vs_neg_deseq2.csv")
pre_df_ctrl["Ch2 KO with DOX+ (PELO KD) vs DOX-"].to_csv("outputs/Ch2_KO_DOXpos_vs_neg_gsea_prerank.csv")

results_focad['FOCAD KO with DOX+ (PELO KD) vs DOX-'].to_csv("outputs/FOCAD_KO_DOXpos_vs_neg_deseq2.csv")
pre_df_focad['FOCAD KO with DOX+ (PELO KD) vs DOX-'].to_csv("outputs/FOCAD_KO_DOXpos_vs_neg_gsea_prerank.csv")